In [4]:
from ipynb.fs.full.common import *

In [102]:
ACTIVE_SHAM = 'Sham'
GROUP1 = 'PRE-AVG'
GROUP2= 'POST-AVG'
COMPUTE_DATA = True
mapping = {0: 'theta', 1: 'alpha', 2:'beta'}
FREQ_BANDS = {
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
}
# PATIENTS = ['Hemlata', 'Malti', 'Preeti', 'Sharifa', 'Vinita'] if ACTIVE_SHAM == 'Active' else [] # ['Geeta', 'Jitendra', 'Jyoti', 'Kuldeep', 'Seema', 'VijayLaxmi']

# COMPUTE DATA

In [8]:
if COMPUTE_DATA:
    if GROUP1 in ['PRE-AVG'] and GROUP2 in ['POST-AVG']:
        folder1 = '2Pre-TDCS'
        folder2 = '4Post-TDCS'

PLI computations

In [9]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


Group 1 data

In [103]:
if COMPUTE_DATA:
    c = 0
    con_mat_1 = {'theta': [], 'alpha': [], 'beta': []}
    for ELECTRODES in ['32electrodes']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder1)
        for patient in os.listdir(folder_path):
            print(patient)
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_1 = data_transformation_easy(_)
                raw_1 = g1_preprocess(raw_1)

                # Epoching
                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    # Manually created events
                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_1):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_1.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_1, i)
                    con_mat_1[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_1)

Kuldeep


Creating RawArray with float64 data, n_channels=32, n_times=74999
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:254: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


...

5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:254: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing

Group 2 data

In [104]:
if COMPUTE_DATA:
    c = 0
    con_mat_2 = {'theta': [], 'alpha': [], 'beta': []}
    for ELECTRODES in ['32electrodes']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder2)
        for patient in os.listdir(folder_path):
            print(patient)
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_2 = data_transformation_easy(_)
                raw_2 = g1_preprocess(raw_2)

                # Epoching
                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    # Manually created events
                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_2):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_2.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_2, i)
                    con_mat_2[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_2)

Kuldeep
Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:254: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing

...

5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:254: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing

Average both con_mat_1 and con_mat_2 and Make matrix symmetric

In [105]:
for i in mapping:
    con_mat_1[mapping[i]] = np.mean(con_mat_1[mapping[i]], axis=0)
    con_mat_2[mapping[i]] = np.mean(con_mat_2[mapping[i]], axis=0)

print(len(con_mat_1), con_mat_1, len(con_mat_1['theta']))
print(len(con_mat_2), con_mat_2, len(con_mat_2['theta']))

3 {'theta': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.4469697 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31013258, 0.40530303, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.23011364, 0.35227273, 0.3219697 , ..., 0.        , 0.        ,
        0.        ],
       [0.3219697 , 0.4436553 , 0.31344697, ..., 0.3532197 , 0.        ,
        0.        ],
       [0.4469697 , 0.25804924, 0.42518939, ..., 0.51988636, 0.57528409,
        0.        ]]), 'alpha': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.37878788, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35340909, 0.4       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.34204545, 0.31325758, 0.4125    , ..., 0.        , 0.        ,
        0.        ],
       [0.356

In [106]:
for i in mapping:
    for row in range(n_channels):
        for col in range(n_channels):
            con_mat_1[mapping[i]][row][col] = con_mat_1[mapping[i]][col][row]
            con_mat_2[mapping[i]][row][col] = con_mat_2[mapping[i]][col][row]

### 3. Thresholding - M1 -  Small World Index

In [ ]:
# Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
# Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [109]:
optimal_threshold = {}
edges_1 = {}
edges_2 = {}
# Generating 50 random networks with same number of vertices and edges 
random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
thresholds = np.arange(0.1, 0.9, 0.002)
table = []

for i in mapping:
    edges_1[mapping[i]] = []
    edges_2[mapping[i]] = []
    ratios = []
    for threshold in thresholds:
        # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
        g1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]] > threshold))
        Lw_1_binarized, CC_1_binarized = g1[0], g1[3]

        g2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]] > threshold))
        Lw_2_binarized, CC_2_binarized = g2[0], g2[3]

        swi_ratio_binarized =  (CC_1_binarized*Lw_2_binarized)/(Lw_1_binarized*CC_2_binarized) if Lw_1_binarized and CC_2_binarized else 0
        ratios.append(swi_ratio_binarized)
        
    optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))]
    t_stat, p_value = ttest_ind(con_mat_1[mapping[i]].flatten(), con_mat_2[mapping[i]].flatten())
    if p_value < 0.005: # Reject the null hypotheses
        Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
        Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

        graph1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]] > optimal_threshold[mapping[i]]))
        Lw_1_binarized, CC_1_binarized = graph1[0], graph1[3]

        graph2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]] > optimal_threshold[mapping[i]]))
        Lw_2_binarized, CC_2_binarized = graph2[0], graph2[3]

        swi_binarized_1 =  (CC_1_binarized/Crand)/(Lw_1_binarized/Lrand)
        swi_binarized_2 = (CC_2_binarized/Crand)/(Lw_2_binarized/Lrand)
        print(swi_binarized_1, "Cc1 ", CC_1_binarized, "Lw1 ", Lw_1_binarized)
        print(swi_binarized_2, "Cc2 ", CC_2_binarized, "Lw2 ", Lw_2_binarized)  

        edges_1[mapping[i]].append(nx.from_numpy_array(con_mat_1[mapping[i]] > optimal_threshold[mapping[i]]).number_of_edges())
        edges_2[mapping[i]].append(nx.from_numpy_array(con_mat_2[mapping[i]] > optimal_threshold[mapping[i]]).number_of_edges())

        print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]} (Significant)", p_value)
        table.append([mapping[i], str(swi_binarized_1)+' ('+str(CC_1_binarized)+','+str(Lw_1_binarized)+') ',  str(swi_binarized_2)+' ('+str(CC_2_binarized)+','+str(Lw_2_binarized)+') '])
    else:
        table.append([mapping[i], None, None])

1.0 Cc1  1.0 Lw1  1.0
0.9959758551307841 Cc2  0.9979838709677413 Lw2  1.002016129032258
theta - Optimal Threshold: 0.14800000000000005 (Significant) 1.303241221095339e-38
0.13011695906432746 Cc1  0.018178104575163397 Lw1  0.13970588235294118
0.006012709137709136 Cc2  0.002607709750566893 Lw2  0.4336996336996337
alpha - Optimal Threshold: 0.6460000000000005 (Significant) 0.004464784329665279
0.13994708994708996 Cc1  0.025253610216016232 Lw1  0.18045112781954886
0.1317391027340007 Cc2  0.09893051393051393 Lw2  0.7509578544061303
beta - Optimal Threshold: 0.6020000000000004 (Significant) 1.0597310470949867e-31


In [80]:
df = pd.DataFrame(table, columns=['FREQ BAND', f'{GROUP1}-CC/Lw', f'{GROUP2}-CC/Lw'])
df.to_excel('comparison-results/average-swi.xlsx')

In [111]:
optimal_threshold

{'theta': 0.14800000000000005,
 'alpha': 0.6460000000000005,
 'beta': 0.6020000000000004}

In [110]:
binarized_matrix = {}
for i in mapping:
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = con_mat_1[mapping[i]] > optimal_threshold[mapping[i]]
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = con_mat_2[mapping[i]] > optimal_threshold[mapping[i]]
print(binarized_matrix.keys())

dict_keys(['PRE-AVG-theta', 'POST-AVG-theta', 'PRE-AVG-alpha', 'POST-AVG-alpha', 'PRE-AVG-beta', 'POST-AVG-beta'])


Find number of links in each graph

In [112]:
table = []
for i in mapping:
    links_g1 = 0
    links_g2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_g1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_g2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_g1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_g2)

    table.append([mapping[i], links_g1, links_g2])

df = pd.DataFrame(table, columns=['FREQ BAND', f'{GROUP1}-Links', f'{GROUP2}-Links'])
df.to_excel('comparison-results/average-links.xlsx')

PRE-AVG-theta-number of links: K= 496
POST-AVG-theta-number of links: K= 495
PRE-AVG-alpha-number of links: K= 22
POST-AVG-alpha-number of links: K= 19
PRE-AVG-beta-number of links: K= 35
POST-AVG-beta-number of links: K= 90


### 5. Asymmetry in  each band

In [113]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

table = []
for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    table.append([mapping[i], lr/rr, lrc/rrc])

df = pd.DataFrame(table, columns=['FREQ BAND', f'{GROUP1}-Left/Right edges', f'{GROUP2}-Left/Right edges'])
df.to_excel('comparison-results/average-asymmetry.xlsx')

PRE-AVG:  theta left region:  344 right region:  344 proportion:  1.0
POST-AVG:  theta left region:  343 right region:  344 proportion:  0.997093023255814
PRE-AVG:  alpha left region:  20 right region:  8 proportion:  2.5
POST-AVG:  alpha left region:  14 right region:  13 proportion:  1.0769230769230769
PRE-AVG:  beta left region:  29 right region:  13 proportion:  2.230769230769231
POST-AVG:  beta left region:  72 right region:  53 proportion:  1.3584905660377358


### 6. Difference matrix

In [114]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [115]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [116]:
# ALL_CHANNELS = np.array(['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2',
#        'Fp1', 'AF3', 'Fz', 'FC2', 'Cz', 'CP2', 'PO3', 'O1', 'Oz', 'O2',
#        'PO4', 'Pz', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5',
#        'T7', 'P7'])
# def set_montage(raw):
#     mont1020 = mne.channels.make_standard_montage('standard_1020')
#     ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in ALL_CHANNELS]
#     mont1020_new = mont1020.copy()
#     mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
#     kept_channel_info = [mont1020.dig[x+3] for x in ind]
#     mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
#     print("sjkghdkgu", mont1020_new)
#     return raw.set_montage(mont1020_new)

In [125]:
for i in mapping:
    plt.imshow(binarized_matrix[f'{GROUP2}-{mapping[i]}'], interpolation='none')
    plt.title(f'Binary Functional connectivity - POST- {[mapping[i]]} - Visualization')
    plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

In [128]:
difference_matrix = {}
for i in mapping:
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) 
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    # # Yellow - Higher synchronisation
    plt.imshow(difference_matrix[mapping[i]], interpolation='none')
    plt.title(f'Binary Functional connectivity difference- {[mapping[i]]} - Visualization')
    plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

In [142]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []

    for ch in range(32):
        # TODO: instead of number of edges wise, we can explore other algorithm
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > 5:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

{'theta': [], 'alpha': ['O1'], 'beta': ['F4', 'FC2', 'PO3', 'O1', 'PO4', 'C3', 'FC5', 'CP5', 'P7']}


Create dictionary of Regions in brain vs channels

In [143]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [144]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

print(selected_regions)

{'theta': [], 'alpha': ['LPO'], 'beta': ['RC', 'RF', 'LPO', 'RPO', 'LC']}


### 7. Network Metrics for selected regions for selected bands


Boxplot helps in reading median, interquartile range and outliers
Box : middle 50% of all data lies (Interquartile range)
Lower end: 1st quartile
Upper end: 3rd quartile
Solid line: Median
Dashed line: Mean
T shaped whiskers: Last point 1.5 times the interquartile range (max and min without outliers)
Points: Further out are outliers

In [ ]:
group_measures = ['Lw', 'NBC', 'Eglo', 'CC', 'Eloc']

In [ ]:
# # Integration metrics - Lavg, Eglo 
# # Segregation metrics - CC, Eloc, Nbc
# import seaborn as sns
# for i in mapping:
#     selected_region = selected_regions[mapping[i]]

#     if selected_region == []:
#         continue

#     # con_MDD = connectivity_matrix(epochs_1, i)
#     # con_control = connectivity_matrix(epochs_2, i)
#     G_MDD = nx.from_numpy_array(binarized_matrix[f'{GROUP1}-{mapping[i]}'])
#     G_Control = nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}'])

#     for region in selected_region:
#         channels_list = regionvschannel[region]
#         G_sub = G_MDD.subgraph(return_index(channels_list, raw_1))
#         MDD_list = calculate_avergae_components(G_sub)
#         G_sub = G_Control.subgraph(return_index(channels_list, raw_2))
#         control_list = calculate_avergae_components(G_sub)

#         print(MDD_list, "fdhgjgdfjd", control_list)

#         # CHECK HERE if keys in MDD_list[1] matches with keys in control_list[1]
#         # and keys in MDD_list[3] matches with keys in control_list[3]
        
#         values_MDD = []
#         values_control = []
#         group_labels_MDD = []
#         group_labels_control = [] 

#         for _ in range(5):
#             print("********", MDD_list)
#             if len(MDD_list[_]):
#                 if type(MDD_list[_][0]) == type(dict()):
#                     x = []
#                     for __ in range(len(MDD_list[_])):
#                         for k,v in MDD_list[_][__].items():
#                             x.append(v)
#                     values_MDD.append(x)
#                 else:
#                     values_MDD.append(MDD_list[_])
#             else:
#                 values_MDD.append([])
#             if (control_list[_]):
#                 if type(control_list[_][0]) == type(dict()):
#                     y = []
#                     for __ in range(len(control_list[_])):
#                         for k,v in control_list[_][__].items():
#                             y.append(v)
#                     values_control.append(y)
#                 else:
#                     values_control.append(control_list[_])
#             else:
#                 values_control.append([])
#             group_labels_MDD.append(f'{GROUP1}-{group_measures[_]}')
#             group_labels_control.append(f'{GROUP2}-{group_measures[_]}')
            
#         values_all = values_1 + values_control
#         group_labels = group_labels_MDD + group_labels_control

#         print("values all", values_all)
#         print("group labels", group_labels)

#         plt.figure(figsize=(12, 8))
#         plt.boxplot(values_all, labels=group_labels, vert=True)
#         plt.title(f'Boxplot of Network Metrics for MDD and Control Groups in {mapping[i]} band in {region} region')
#         plt.xlabel('Network Metrics')
#         plt.ylabel('Values')
#         plt.show()

8. Mark all regions in brain along with Netowrk metrics which are significant